In [7]:
import pandas as pd


In [8]:
def bcf_to_twh(bcf, conversion_factor=0.293):
    """
    Converts natural gas volume from BCF to TWh.
    
    Parameters:
    - bcf (float): volume in billion cubic feet
    - conversion_factor (float): standard conversion factor (default: 0.293 TWh/BCF)
    
    Returns:
    - float: energy in TWh
    """
    return round(bcf * conversion_factor, 4)



In [ ]:

# Natural Gas Production
raw_gas_production_df = pd.read_csv("https://ourworldindata.org/grapher/gas-production-by-country.csv?v=1&csvType=filtered&useColumnShortNames=true&time=earliest..2024&country=~COL&overlay=download-data", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})

# Natural Gas Consumption
raw_gas_consumption_df = pd.read_csv("https://ourworldindata.org/grapher/gas-consumption-by-country.csv?v=1&csvType=filtered&useColumnShortNames=true&tab=line&country=~COL&mapSelect=~COL&overlay=download-data", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})

# Natural Gas Trade (bcf)
raw_gas_trade_df = pd.read_csv("./data/raw/INT-Export-10-21-2025_21-19-22.csv", skiprows=1)

# Energy Consumption by Source
#raw_source_consumtion_df = pd.read_csv("https://ourworldindata.org/grapher/primary-sub-energy-source.csv?v=1&csvType=filtered&useColumnShortNames=true&country=~COL&overlay=download-data", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})

# Electricity Production by Sorce
#raw_source_production_df = pd.read_csv("https://ourworldindata.org/grapher/electricity-prod-source-stacked.csv?v=1&csvType=filtered&useColumnShortNames=true&country=~COL&overlay=download-data", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})

# Electricity Production by Sources in Percentages
raw_electricity_production_df = pd.read_csv("https://ourworldindata.org/grapher/share-elec-by-source.csv?v=1&csvType=filtered&useColumnShortNames=true&country=~COL&overlay=download-data")


C:\Users\keato\AppData\Local\Temp\ipykernel_22256\1152028477.py:20: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_gas_prices_df = pd.read_csv("./data/raw/gas_prices.csv", skiprows=1)


In [10]:
gas_trade_df = raw_gas_trade_df.iloc[[2,4],2:].T.reset_index()
gas_trade_df.columns = ["Year", "imports", "exports"]
gas_trade_df.fillna(0, inplace=True)
gas_trade_df.Year = gas_trade_df.Year.astype(int)

In [11]:
rows_to_add = [{"Year": year, "imports": 0, "exports": 0} for year in range(1965, 1980)]

In [12]:
gas_trade_df = pd.concat([gas_trade_df, pd.DataFrame(rows_to_add)], ignore_index=True).sort_values(by="Year").reset_index(drop=True)
gas_trade_df.loc[len(gas_trade_df)] = [2024, 78.1465, 0.0] # Adding data for 2024, taken from here: https://www.elcolombiano.com/negocios/importacion-de-gas-natural-en-colombia-aumento-en-2024-pero-produccion-cae-HP27286357

gas_trade_df["imports_twh"] = gas_trade_df.imports.apply(bcf_to_twh)
gas_trade_df["exports_twh"] = gas_trade_df.exports.apply(bcf_to_twh)
gas_trade_df.drop(columns=["imports", "exports"], inplace=True)

In [13]:
gas_production_df = raw_gas_production_df[raw_gas_production_df["Year"] >= 1965 ][["Year", "gas_production__twh"]].copy()
gas_consumption_df = raw_gas_consumption_df[raw_gas_consumption_df["Year"] >= 1965 ][["Year", "gas_consumption_twh"]].copy()

gas_balance = gas_production_df.merge(gas_consumption_df, on="Year").merge(gas_trade_df, on="Year")

In [14]:
gas_balance

,Year,gas_production__twh,gas_consumption_twh,imports_twh,exports_twh
0,1965,11.378318,8.951250,0.0000,0.0000
1,1966,13.802176,9.336250,0.0000,0.0000
2,1967,14.455236,11.550000,0.0000,0.0000
3,1968,15.233884,11.771375,0.0000,0.0000
4,1969,16.803740,12.854188,0.0000,0.0000
5,1970,12.493251,12.493251,0.0000,0.0000
6,1971,13.004500,13.004500,0.0000,0.0000
7,1972,15.885500,15.885250,0.0000,0.0000
8,1973,16.460000,16.460000,0.0000,0.0000
9,1974,17.125500,17.125750,0.0000,0.0000


In [15]:
gas_balance.to_csv("./data/processed/gas_balance.csv", index=False)

In [16]:
electricity_production_df = raw_electricity_production_df.iloc[:, [2,4,5]]
electricity_production_df = electricity_production_df.fillna(0)

In [17]:
electricity_production_df.to_csv("./data/processed/electricity_production.csv", index=False)

# Predictions
